<h1>Toronto Segmenting & Clustering based on Food Venues</h1>

Installing the and importing the required libraries:

In [1]:
!pip install lxml html5lib beautifulsoup4

In [2]:
import pandas as pd

<h2>Web scraping</h2>

In [19]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
scraped_data = pd.read_html(url)
scrape_df = scraped_data[0]
scrape_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h3>Drop unassigned Boroughs

In [20]:
indexes = scrape_df[scrape_df['Borough'] == "Not assigned"].index
scrape_df.drop(indexes, inplace=True)
scrape_df = scrape_df.reset_index(drop=True)
scrape_df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h3>Check for duplicates in postal codes

In [21]:
duplicateDFRow = scrape_df[scrape_df.duplicated(['Postal Code'])]
print(duplicateDFRow)

Empty DataFrame
Columns: [Postal Code, Borough, Neighborhood]
Index: []


<h3>Check for unassigned Neighborhoods

In [22]:
uneigh = scrape_df[scrape_df['Neighborhood'] == "Not assigned"].index
uneigh

Int64Index([], dtype='int64')

In [25]:
scrape_df.shape

(103, 3)

Since there are no duplicates in postal codes and neighborhoods, no further processing is required at this point.

<h3>We will proceed to add in the latitude and longitude from a separate CSV file.</h3>

In [28]:
gurl = "https://cocl.us/Geospatial_data"
spatial_df = pd.read_csv(gurl)
spatial_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [55]:
df = pd.merge(scrape_df, spatial_df, on='Postal Code')
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


The two DFs are now merged into one which has the postal code, borough, neighborhood name(s) and latlong coordinates.
<h3> Now dependencies for the map and clustering:</h3>
Installing dependencies

In [49]:
!pip install folium
import folium 
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
print("Execution completed")

Execution completed


<h3>Generating the map of neighborhoods

In [39]:
YTO_lat = "43.7184038"
YTO_long = "-79.5181407"
#no point engaging an API just to get the latitude/longitude of one location (Toronto). I just grabbed it from google maps.

In [95]:
map_toronto = folium.Map(location=[YTO_lat, YTO_long], zoom_start=11)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h2>Connecting to Foursquare API

In [44]:
# The code was removed by Watson Studio for sharing.

In [50]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [51]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL. Please note Category ID is already entered as we are only interested in food places.
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId=4d4b7105d754a06374d81259'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [56]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

<h3>Transforming the data for analysis

In [57]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
2,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant
3,Parkwoods,43.753259,-79.329656,High Street Fish & Chips,43.745260,-79.324949,Fish & Chips Shop
4,Parkwoods,43.753259,-79.329656,Pizza Pizza,43.760231,-79.325666,Pizza Place


In [58]:
toronto_venues.shape

(3750, 7)

<h4>Grouping the venue categories together by neighborhoods to count

In [59]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,42,42,42,42,42,42
"Alderwood, Long Branch",5,5,5,5,5,5
"Bathurst Manor, Wilson Heights, Downsview North",11,11,11,11,11,11
Bayview Village,5,5,5,5,5,5
"Bedford Park, Lawrence Manor East",27,27,27,27,27,27
Berczy Park,100,100,100,100,100,100
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",97,97,97,97,97,97
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",29,29,29,29,29,29


In [61]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 113 unique categories.


<h3>One-hot encoding to ensure data can be used and parsed for analysis</h3>
    <p>This is because we want to calculate the means of each food store category that appears, so we can rank them.

In [62]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
toronto_onehot.shape

(3750, 114)

In [65]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Agincourt,0.00,0.0,0.023810,0.0,0.0,0.047619,0.023810,0.000000,0.047619,...,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.023810,0.000000
1,"Alderwood, Long Branch",0.00,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.00,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,Bayview Village,0.00,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,"Bedford Park, Lawrence Manor East",0.00,0.0,0.037037,0.0,0.0,0.000000,0.000000,0.037037,0.037037,...,0.000000,0.0,0.074074,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.037037
5,Berczy Park,0.00,0.0,0.040000,0.0,0.0,0.010000,0.020000,0.010000,0.030000,...,0.000000,0.0,0.010000,0.00,0.000000,0.000000,0.0,0.020000,0.000000,0.000000
6,"Birch Cliff, Cliffside West",0.00,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.250000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.00,0.0,0.010309,0.0,0.0,0.010309,0.010309,0.000000,0.061856,...,0.010309,0.0,0.010309,0.00,0.030928,0.000000,0.0,0.030928,0.010309,0.000000
8,"Business reply mail Processing Centre, South C...",0.00,0.0,0.034483,0.0,0.0,0.000000,0.034483,0.000000,0.068966,...,0.000000,0.0,0.034483,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [67]:
toronto_grouped.shape

(97, 114)

<h3>Now we rank the top 10 food places by their means for each venue, so that we can start the clustering analysis.

In [68]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [69]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Bakery,Sandwich Place,Restaurant,Asian Restaurant,Pizza Place,Caribbean Restaurant,Shanghai Restaurant,Seafood Restaurant,Cantonese Restaurant
1,"Alderwood, Long Branch",Pizza Place,Sandwich Place,Donut Shop,Moroccan Restaurant,Wings Joint,Fish & Chips Shop,Diner,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Pizza Place,Mediterranean Restaurant,Sushi Restaurant,Diner,Restaurant,Chinese Restaurant,Sandwich Place,Middle Eastern Restaurant,Deli / Bodega,Fried Chicken Joint
3,Bayview Village,Japanese Restaurant,Restaurant,Chinese Restaurant,Café,Wings Joint,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Pizza Place,Sandwich Place,Thai Restaurant,Sushi Restaurant,Restaurant,Wings Joint,American Restaurant,Bagel Shop,Bakery


<h3>Running the K-means clustering

In [70]:
# set number of clusters
kclusters = 5

toronto_grouped_clusters = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clusters)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 1, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [71]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Pizza Place,Chinese Restaurant,Caribbean Restaurant,Fast Food Restaurant,Café,Fish & Chips Shop,Wings Joint,Dim Sum Restaurant,Doner Restaurant,Donut Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Pizza Place,French Restaurant,Café,Portuguese Restaurant,Wings Joint,Filipino Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2.0,Café,Diner,Sandwich Place,Pizza Place,Sushi Restaurant,Bakery,Restaurant,Italian Restaurant,Thai Restaurant,Mediterranean Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2.0,Fast Food Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Italian Restaurant,Café,Korean Restaurant,Greek Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2.0,Japanese Restaurant,Café,Sushi Restaurant,Burger Joint,Restaurant,Italian Restaurant,Sandwich Place,Middle Eastern Restaurant,Ramen Restaurant,Gastropub


In [83]:
toronto_merged.loc[toronto_merged['Cluster Labels'].isnull()]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,M9W,Etobicoke,"Northwest, West Humber - Clairville",43.706748,-79.594054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h4>It seems that there are actually 2 neighborhoods with no food places within 1000 metres! I wouldn't want to live there... But we can just remove them from the clustering analysis as they are essentially outliers.

In [86]:
toronto_merged = toronto_merged.dropna()
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(np.uint8)

<h2>Visualizing the results

In [87]:
# create map
map_clusters = folium.Map(location=[YTO_lat, YTO_long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [89]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,0,Pizza Place,Bakery,Fast Food Restaurant,Café,Wings Joint,Food Court,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
8,East York,0,Pizza Place,Bakery,Fast Food Restaurant,Restaurant,Gastropub,Breakfast Spot,Café,Fish & Chips Shop,Diner,Doner Restaurant
10,North York,0,Italian Restaurant,Pizza Place,Restaurant,Mediterranean Restaurant,Latin American Restaurant,Asian Restaurant,Sushi Restaurant,Fast Food Restaurant,Japanese Restaurant,Sandwich Place
12,Scarborough,0,Breakfast Spot,Italian Restaurant,Burger Joint,Wings Joint,Food Truck,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
26,Scarborough,0,Bakery,Indian Restaurant,Chinese Restaurant,Pizza Place,Asian Restaurant,Burger Joint,Caribbean Restaurant,Fried Chicken Joint,German Restaurant,Hakka Restaurant
50,North York,0,Pizza Place,Italian Restaurant,Bakery,Wings Joint,Food,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
57,North York,0,Bakery,Donut Shop,Italian Restaurant,Wings Joint,Food Court,Diner,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
72,North York,0,Pizza Place,Eastern European Restaurant,Bakery,Sushi Restaurant,Wings Joint,Food,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
98,Etobicoke,0,Pizza Place,Italian Restaurant,Burger Joint,Breakfast Spot,Sushi Restaurant,Bakery,French Restaurant,Chinese Restaurant,Restaurant,Seafood Restaurant
101,Etobicoke,0,Italian Restaurant,Poutine Place,Eastern European Restaurant,Latin American Restaurant,Food,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [90]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,1,Pizza Place,Fish & Chips Shop,Restaurant,Café,Mexican Restaurant,Wings Joint,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
14,East York,1,Pizza Place,Thai Restaurant,Sandwich Place,Diner,Greek Restaurant,Restaurant,Café,Fast Food Restaurant,Deli / Bodega,Dim Sum Restaurant
17,Etobicoke,1,Pizza Place,Fish & Chips Shop,Sandwich Place,Café,Breakfast Spot,Wings Joint,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
18,Scarborough,1,Pizza Place,Restaurant,Greek Restaurant,Fried Chicken Joint,Sandwich Place,Chinese Restaurant,Fast Food Restaurant,Filipino Restaurant,Breakfast Spot,Mexican Restaurant
32,Scarborough,1,Pizza Place,Restaurant,Sandwich Place,Japanese Restaurant,Wings Joint,Filipino Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
34,North York,1,Pizza Place,Restaurant,American Restaurant,Doner Restaurant,Vietnamese Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Chinese Restaurant,Sandwich Place,Sushi Restaurant
51,Scarborough,1,Pizza Place,Burger Joint,Wings Joint,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
56,York,1,Pizza Place,Restaurant,Italian Restaurant,Sandwich Place,Wings Joint,Filipino Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
63,York,1,Pizza Place,Sandwich Place,Indian Restaurant,BBQ Joint,Fast Food Restaurant,Café,Fried Chicken Joint,Burger Joint,Thai Restaurant,Asian Restaurant
64,York,1,Pizza Place,Middle Eastern Restaurant,Diner,Sandwich Place,Fried Chicken Joint,Breakfast Spot,Café,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [91]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,2,Pizza Place,French Restaurant,Café,Portuguese Restaurant,Wings Joint,Filipino Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
2,Downtown Toronto,2,Café,Diner,Sandwich Place,Pizza Place,Sushi Restaurant,Bakery,Restaurant,Italian Restaurant,Thai Restaurant,Mediterranean Restaurant
3,North York,2,Fast Food Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Italian Restaurant,Café,Korean Restaurant,Greek Restaurant
4,Downtown Toronto,2,Japanese Restaurant,Café,Sushi Restaurant,Burger Joint,Restaurant,Italian Restaurant,Sandwich Place,Middle Eastern Restaurant,Ramen Restaurant,Gastropub
6,Scarborough,2,Fast Food Restaurant,Restaurant,Chinese Restaurant,African Restaurant,Greek Restaurant,Caribbean Restaurant,Bakery,Sandwich Place,Wings Joint,Falafel Restaurant
7,North York,2,Restaurant,Japanese Restaurant,Asian Restaurant,Burger Joint,Pizza Place,Café,Deli / Bodega,Sandwich Place,Middle Eastern Restaurant,Dim Sum Restaurant
9,Downtown Toronto,2,Japanese Restaurant,Gastropub,Italian Restaurant,Café,Diner,Restaurant,American Restaurant,Ramen Restaurant,Seafood Restaurant,Burger Joint
13,North York,2,Restaurant,Japanese Restaurant,Asian Restaurant,Burger Joint,Pizza Place,Café,Deli / Bodega,Sandwich Place,Middle Eastern Restaurant,Dim Sum Restaurant
15,Downtown Toronto,2,Café,Restaurant,Italian Restaurant,Breakfast Spot,Seafood Restaurant,Bakery,Japanese Restaurant,Gastropub,American Restaurant,Diner
16,York,2,Pizza Place,Bagel Shop,Sushi Restaurant,Restaurant,Mexican Restaurant,Burger Joint,Deli / Bodega,Café,Gastropub,Sandwich Place


In [92]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,3,Deli / Bodega,Cafeteria,Wings Joint,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant


In [93]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4,Pizza Place,Chinese Restaurant,Caribbean Restaurant,Fast Food Restaurant,Café,Fish & Chips Shop,Wings Joint,Dim Sum Restaurant,Doner Restaurant,Donut Shop
22,Scarborough,4,Indian Restaurant,Pizza Place,Chinese Restaurant,Fast Food Restaurant,Food,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
27,North York,4,Pizza Place,Restaurant,Sandwich Place,Chinese Restaurant,Bakery,Korean Restaurant,Fast Food Restaurant,Filipino Restaurant,Dim Sum Restaurant,Diner
38,Scarborough,4,Chinese Restaurant,Pizza Place,Asian Restaurant,Sandwich Place,Fast Food Restaurant,Burger Joint,Hakka Restaurant,Greek Restaurant,Diner,Doner Restaurant
49,North York,4,Pizza Place,Chinese Restaurant,Bakery,Mediterranean Restaurant,Dim Sum Restaurant,Wings Joint,Food,Doner Restaurant,Donut Shop,Dumpling Restaurant
77,Etobicoke,4,Pizza Place,American Restaurant,Sandwich Place,Chinese Restaurant,Wings Joint,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
78,Scarborough,4,Chinese Restaurant,Bakery,Sandwich Place,Restaurant,Asian Restaurant,Pizza Place,Caribbean Restaurant,Shanghai Restaurant,Seafood Restaurant,Cantonese Restaurant
85,Scarborough,4,Chinese Restaurant,Pizza Place,Noodle House,Korean Restaurant,Bakery,BBQ Joint,Fast Food Restaurant,Fried Chicken Joint,Japanese Restaurant,Udon Restaurant
90,Scarborough,4,Chinese Restaurant,Fast Food Restaurant,Pizza Place,Bakery,Indian Restaurant,Breakfast Spot,Japanese Restaurant,Noodle House,Sandwich Place,Burger Joint


<h3>Thanks for reviewing my work!